In [ ]:
# Mengimpor pustaka yang diperlukan untuk analisis dan prapemrosesan data.
# - pandas untuk manipulasi data.
# - numpy untuk operasi numerik.
# - sklearn.model_selection untuk membagi data menjadi set pelatihan dan pengujian.
# - sklearn.preprocessing untuk pengkodean fitur dan penskalaan.
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

In [ ]:
# Membaca dataset dari file CSV.
df = pd.read_csv('../Data/data_1.csv')

# Menampilkan informasi dasar tentang DataFrame.
# Ini termasuk jumlah entri, jumlah kolom, nama kolom, jumlah nilai non-null, dan tipe data setiap kolom.
df.info()

# Menampilkan lima baris pertama dari DataFrame untuk mendapatkan gambaran umum tentang data.
df.head()

C:\Users\albia\AppData\Local\Temp\ipykernel_1348\808544169.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Data/data_1.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 35 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   pkSeqID       1000000 non-null  int64  
 1   stime         1000000 non-null  float64
 2   flgs          1000000 non-null  object 
 3   proto         1000000 non-null  object 
 4   saddr         1000000 non-null  object 
 5   sport         999513 non-null   object 
 6   daddr         1000000 non-null  object 
 7   dport         999513 non-null   object 
 8   pkts          1000000 non-null  int64  
 9   bytes         1000000 non-null  int64  
 10  state         1000000 non-null  object 
 11  ltime         1000000 non-null  float64
 12  seq           1000000 non-null  int64  
 13  dur           1000000 non-null  float64
 14  mean          1000000 non-null  float64
 15  stddev        1000000 non-null  float64
 16  smac          0 non-null        float64
 17  dmac          0 non-null    

,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,bytes,...,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category,subcategory
0,1,1.526344e+09,e,arp,192.168.100.1,NaN,192.168.100.3,NaN,4,240,...,2,2,120,120,0.002508,0.000836,0.000836,0,Normal,Normal
1,2,1.526344e+09,e,tcp,192.168.100.7,139,192.168.100.4,36390,10,680,...,5,5,350,330,0.006190,0.002751,0.002751,0,Normal,Normal
2,3,1.526344e+09,e,udp,192.168.100.149,51838,27.124.125.250,123,2,180,...,1,1,90,90,20.590960,0.000000,0.000000,0,Normal,Normal
3,4,1.526344e+09,e,arp,192.168.100.4,NaN,192.168.100.7,NaN,10,510,...,5,5,210,300,0.006189,0.002751,0.002751,0,Normal,Normal
4,5,1.526344e+09,e,udp,192.168.100.27,58999,192.168.100.1,53,4,630,...,2,2,174,456,0.005264,0.001755,0.001755,0,Normal,Normal


In [ ]:
# Menghapus spasi di awal atau akhir dari setiap nama kolom.
# Langkah ini penting untuk mencegah kesalahan saat mengakses kolom.
df.columns = df.columns.str.strip()

In [ ]:
# Menghapus kolom yang tidak relevan atau yang seluruhnya berisi nilai null.
# Kolom-kolom ini ('smac', 'dmac', 'soui', 'doui', 'sco', 'dco') tidak memberikan informasi yang berguna untuk analisis.
df.drop(columns=["smac", "dmac", "soui", "doui", "sco", "dco"], inplace=True)

In [ ]:
# Menghapus baris-baris duplikat dari DataFrame.
# Ini memastikan bahwa setiap baris dalam data adalah unik.
df.drop_duplicates(inplace=True)

In [ ]:
# Menghapus baris yang memiliki nilai null di kolom 'sport' atau 'dport'.
# Baris dengan data yang hilang tidak dapat digunakan untuk pembuatan model.
df.dropna(subset=["sport", "dport"], inplace=True)

In [ ]:
# Memisahkan fitur (variabel independen) dan target (variabel dependen).
# 'attack' adalah variabel target yang akan diprediksi.
target = "attack"  
X = df.drop(columns=[target])
y = df[target]

In [ ]:
# Menampilkan jumlah kemunculan setiap nilai unik dalam variabel target.
# Ini berguna untuk memahami distribusi kelas (misalnya, seimbang atau tidak seimbang).
print(y.value_counts())

attack
1    997753
0      1760
Name: count, dtype: int64


In [ ]:
# --- Penjelasan Proses Label Encoding untuk Fitur Kardinalitas Tinggi ---
#
# Latar Belakang: Model machine learning pada dasarnya bekerja dengan angka.
# Fitur seperti alamat IP ('saddr', 'daddr') atau nomor port ('sport', 'dport')
# adalah data kategorikal (dalam bentuk teks/objek). Oleh karena itu, kita perlu
# mengubahnya menjadi format numerik.
#
# Mengapa LabelEncoder? Kolom-kolom ini memiliki 'kardinalitas tinggi', yang berarti
# mereka memiliki sangat banyak nilai unik (ribuan alamat IP atau port yang berbeda).
# Jika kita menggunakan OneHotEncoder, kita akan menciptakan ribuan kolom baru,
# yang tidak efisien (disebut 'curse of dimensionality').
#
# LabelEncoder adalah pendekatan yang lebih praktis di sini. Cara kerjanya:
# 1. Ia memindai semua nilai unik dalam sebuah kolom.
# 2. Ia membuat "kamus" pemetaan, di mana setiap nilai unik diberi sebuah angka integer unik (misal: '192.168.1.1' -> 0, '8.8.8.8' -> 1, dst.).
# 3. Ia kemudian mengubah setiap nilai di kolom tersebut sesuai dengan pemetaan yang telah dibuat.

# Mengidentifikasi kolom kategorikal dengan kardinalitas tinggi.
high_card_cat = ["saddr", "daddr", "sport", "dport"]

# Melakukan iterasi untuk menerapkan Label Encoding pada setiap kolom.
for col in high_card_cat:
    # Membuat instance (objek) dari LabelEncoder.
    le = LabelEncoder()
    
    # Menerapkan .fit_transform() pada kolom:
    # - .fit(): Mempelajari semua kategori unik dan membuat pemetaan internal (kamus).
    # - .transform(): Menggunakan pemetaan tersebut untuk mengubah setiap nilai menjadi angka.
    # Hasilnya langsung menggantikan kolom asli dengan versi numeriknya.
    X[col] = le.fit_transform(X[col].astype(str))

In [ ]:
# --- Analisis Distribusi Fitur 'daddr' Setelah Encoding ---
#
# Konteks: Setelah menerapkan LabelEncoder, nilai-nilai asli alamat IP di kolom 'daddr'
# telah digantikan oleh label numerik (misalnya, 26, 29, 32, dll.).
#
# Cara Membaca Output di Bawah:
# Perintah `value_counts()` sekarang menghitung frekuensi kemunculan setiap LABEL NUMERIK, bukan alamat IP asli.
# - Angka di kolom kiri (misal, '26') adalah LABEL yang diberikan oleh encoder.
# - Angka di kolom kanan (misal, '882445') adalah JUMLAH KEMUNCULAN label tersebut.
#
# Dengan kata lain, output "26     882445" berarti:
# "Alamat IP tujuan (daddr) yang paling umum, yang oleh encoder diberi label '26',
# muncul sebanyak 882.445 kali dalam dataset."
#
# Ini membantu kita memahami distribusi lalu lintas ke tujuan yang berbeda, meskipun kita
# melihatnya melalui representasi numerik yang di-encode.

print(X['daddr'].value_counts())

In [ ]:
# Mengidentifikasi kolom kategorikal dengan kardinalitas rendah.
# Kolom-kolom ini memiliki jumlah nilai unik yang relatif sedikit.
low_card_cat = ["proto", "state", "flgs", "category", "subcategory"]

# Membuat instance dari OneHotEncoder.
# sparse_output=False menghasilkan array numpy padat, handle_unknown="ignore" mengabaikan nilai-nilai yang tidak terlihat saat pelatihan.
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

# Menerapkan One-Hot Encoding pada kolom-kolom dengan kardinalitas rendah.
# Proses ini membuat kolom biner baru untuk setiap nilai unik.
ohe_result = ohe.fit_transform(X[low_card_cat])

# Mendapatkan nama-nama kolom baru yang dihasilkan dari One-Hot Encoding.
ohe_cols = ohe.get_feature_names_out(low_card_cat)

# Membuat DataFrame baru dari hasil One-Hot Encoding dengan nama kolom yang sesuai.
X_ohe = pd.DataFrame(ohe_result, columns=ohe_cols, index=X.index)

In [ ]:
# Memilih kolom numerik dari DataFrame fitur (X).
# Ini termasuk semua kolom dengan tipe data 'int64' atau 'float64'.
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Menghapus kolom-kolom yang sudah di-encode (kardinalitas tinggi) dari daftar kolom numerik.
# Hal ini untuk menghindari penskalaan pada kolom yang sudah diubah menjadi numerik melalui Label Encoding.
num_cols = [col for col in num_cols if col not in high_card_cat]

# Membuat instance dari StandardScaler.
# StandardScaler menstandarkan fitur dengan menghilangkan rata-rata dan menskalakan ke varians satuan.
scaler = StandardScaler()

# Menerapkan penskalaan pada kolom-kolom numerik.
scaled_result = scaler.fit_transform(X[num_cols])

# Membuat DataFrame baru dari hasil penskalaan dengan nama kolom yang sesuai.
X_scaled = pd.DataFrame(scaled_result, columns=num_cols, index=X.index)

In [ ]:
# Menggabungkan semua fitur yang telah diproses menjadi satu DataFrame.
# Ini termasuk fitur numerik yang telah diskalakan, fitur kardinalitas tinggi yang di-encode, dan fitur kardinalitas rendah yang di-one-hot-encode.
X_processed = pd.concat([X_scaled, X[high_card_cat], X_ohe], axis=1)

In [ ]:
# Menggabungkan kembali fitur-fitur yang telah diproses dengan variabel target.
final_df = pd.concat([X_processed, y], axis=1)

In [ ]:
# Menampilkan informasi ringkas tentang DataFrame final.
# Ini untuk memverifikasi bahwa semua kolom memiliki tipe data yang benar dan tidak ada nilai null.
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999513 entries, 1 to 999999
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   pkSeqID                   999513 non-null  float64
 1   stime                     999513 non-null  float64
 2   pkts                      999513 non-null  float64
 3   bytes                     999513 non-null  float64
 4   ltime                     999513 non-null  float64
 5   seq                       999513 non-null  float64
 6   dur                       999513 non-null  float64
 7   mean                      999513 non-null  float64
 8   stddev                    999513 non-null  float64
 9   sum                       999513 non-null  float64
 10  min                       999513 non-null  float64
 11  max                       999513 non-null  float64
 12  spkts                     999513 non-null  float64
 13  dpkts                     999513 non-null  float6

In [ ]:
# Memeriksa apakah masih ada nilai yang hilang (null) di dalam DataFrame final.
missing = final_df.isnull().sum()
print(missing[missing > 0])

Series([], dtype: int64)


In [ ]:
# Menampilkan lima baris pertama dari DataFrame final yang telah diproses.
# Ini memberikan gambaran akhir dari data yang siap digunakan untuk pemodelan.
final_df.head()

,pkSeqID,stime,pkts,bytes,ltime,seq,dur,mean,stddev,sum,...,flgs_e dD,flgs_e g,flgs_e r,flgs_e s,flgs_eU,category_Normal,category_Reconnaissance,subcategory_Normal,subcategory_Service_Scan,attack
1,-1.732297,-2.482046,0.004015,-0.009365,-2.474600,-1.148851,38.399038,-0.172964,-0.040363,-0.015883,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
2,-1.732294,-2.482026,-0.012164,-0.010411,-2.482075,-1.148839,-0.041588,0.065206,-0.040468,-0.013431,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
4,-1.732287,-2.481636,-0.008119,-0.009470,-2.478748,-1.148804,15.026018,0.310260,1.010213,-0.005917,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
7,-1.732276,-2.481514,-0.012164,-0.010427,-2.481550,-1.148757,0.023230,-0.173102,-0.040468,-0.015890,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
8,-1.732273,-2.481504,-0.012164,-0.010427,-2.481540,-1.148488,0.023257,-0.173102,-0.040468,-0.015890,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0


In [ ]:
# Menyimpan DataFrame yang telah diproses ke dalam file CSV baru.
# index=False berarti indeks DataFrame tidak akan ditulis ke dalam file.
final_df.to_csv("../Data/Bot_IoT_processed.csv", index=False)

In [ ]:
# Mencetak pesan konfirmasi dan perbandingan bentuk DataFrame sebelum dan sesudah prapemrosesan.
print("Preprocessing selesai. File disimpan sebagai Bot_IoT_processed.csv")
print("Shape sebelum:", df.shape)
print("Shape sesudah:", final_df.shape)

Preprocessing selesai. File disimpan sebagai Bot_IoT_processed.csv
Shape sebelum: (999513, 29)
Shape sesudah: (999513, 53)


In [ ]:
# Mengimpor pustaka joblib untuk menyimpan model atau objek Python.
import joblib

# Menyimpan objek scaler yang telah dilatih ke dalam sebuah file.
# Ini memungkinkan scaler yang sama untuk digunakan kembali nanti, misalnya pada data baru.
joblib.dump(scaler, "../Data/scaler.pkl")

['../Data/scaler.pkl']